In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
import random
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# 1. Laad afbeelding
def load_image(path, size=640):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (size, size))
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
    return img_rgb, img_tensor.unsqueeze(0)

In [ ]:
# 2. Hook op attention-layer
def register_attention_hook(model, layer_name="model.8.m.1.1.attn"):
    attn_maps = []

    def hook_fn(module, input, output):
        attn_maps.append(output)

    target_layer = dict(model.named_modules())[layer_name]
    handle = target_layer.register_forward_hook(hook_fn)
    return attn_maps, handle

In [ ]:
# 3. Visualiseer attention-map
def visualize_attention(attn_map, original_img):
    attn = attn_map.squeeze().detach().cpu().mean(0)
    
    # Veilige normalisatie
    attn_min, attn_max = attn.min(), attn.max()
    if (attn_max - attn_min) > 1e-6:
        attn = (attn - attn_min) / (attn_max - attn_min)
    else:
        attn = torch.zeros_like(attn)

    attn_resized = cv2.resize(attn.numpy(), (original_img.shape[1], original_img.shape[0]))

    heatmap = cv2.applyColorMap(np.uint8(255 * attn_resized), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(original_img, 0.6, heatmap, 0.4, 0)
    
    plt.imshow(overlay)
    plt.axis("off")
    plt.title("YOLOv12 Attention Map")
    plt.show()


In [ ]:
# 4. Hoofdscript
def run_attention_vis(img_path):
    model = YOLO("YOLOv12mv8.pt")  # Zorg dat dit pad klopt
    model.eval()
    img_rgb, img_tensor = load_image(img_path)
    attn_maps, hook = register_attention_hook(model.model)

    _ = model.predict(img_path, imgsz=640, verbose=False)

    hook.remove()
    visualize_attention(attn_maps[0][0], img_rgb)

In [ ]:
# Run op afbeelding
run_attention_vis(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1337.jpg")


In [ ]:
# model = YOLO("YOLOv12mv8.pt")
# for name, _ in model.model.named_modules():
#     if "attn" in name:
#         print(name)



## Gram-cam

In [ ]:
from ultralytics import YOLO

model = YOLO("YOLOv12mv8.pt")  # Vervang dit door het pad naar jouw model
model.eval()


In [ ]:
target_layer = model.model.model[20].m[0].cv3.conv

In [ ]:
import cv2
import numpy as np
import torch

img_path = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1337.jpg"# Vervang dit door het pad naar jouw afbeelding
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, (640, 640))
img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
input_tensor = img_tensor.unsqueeze(0)

In [ ]:
from pytorch_grad_cam.base_cam import BaseCAM
class YOLOBoxTarget:
    def __init__(self, box_index: int):
        self.box_index = box_index

    def __call__(self, model_output):
        # model_output is een tuple; we pakken de box scores (bijv. confidence)
        return model_output[0][self.box_index, 4]  # 4 = confidence column


In [ ]:
results = model.predict(img_path, imgsz=640, verbose=False)
boxes_tensor = results[0].boxes.data  # tensor van [n, 6] (x1, y1, x2, y2, conf, cls)
print(boxes_tensor)

In [ ]:
target = YOLOBoxTarget(box_index=0)  # kies eerste detectie

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

input_tensor.requires_grad_()  # Zorg dat Grad-CAM een backward pass kan doen

target = YOLOBoxTarget(box_index=0)

cam = GradCAM(model=model.model, target_layers=[target_layer])
grayscale_cam = cam(input_tensor=input_tensor, targets=[target])[0]

In [ ]:
from pytorch_grad_cam.utils.image import show_cam_on_image

visualization = show_cam_on_image(img_resized / 255.0, grayscale_cam, use_rgb=True)

import matplotlib.pyplot as plt
plt.imshow(visualization)
plt.axis('off')
plt.title('Grad-CAM uitleg voor YOLOv12')
plt.show()


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torch.nn import Conv2d

# ------------------------
# Afbeelding inladen
# ------------------------
def load_image(path, size=640):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (size, size))
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
    return img_rgb, img_resized, img_tensor.unsqueeze(0)

# ------------------------
# Grad-CAM target op class score
# ------------------------
class ClassScoreTarget:
    def __init__(self, class_index: int):
        self.class_index = class_index

    def __call__(self, model_output):
        class_scores = model_output[0][:, 5:]  # [N, num_classes]
        return class_scores[:, self.class_index].max()

# ------------------------
# Grad-CAM uitvoeren
# ------------------------
def run_grad_cam(img_path):
    # 1. Laad model
    model = YOLO("YOLOv12mv8.pt")
    model.eval()

    # 2. Laad afbeelding
    img_rgb, img_resized, input_tensor = load_image(img_path)
    input_tensor = input_tensor.to(model.device)
    input_tensor.requires_grad_()

    # 3. Predictie uitvoeren om klasse te bepalen
    results = model.predict(img_path, imgsz=640, verbose=False)
    boxes = results[0].boxes.data
    if boxes.shape[0] == 0:
        print("Geen objecten gedetecteerd.")
        return

    class_index = int(boxes[0, 5])
    target = ClassScoreTarget(class_index)
    print(f"Geselecteerde klasse-index: {class_index}")

    # 4. Selecteer automatisch laatste Conv2d-laag
    target_layer = None
    for layer in reversed(model.model.model[:-1]):
        for sublayer in layer.modules():
            if isinstance(sublayer, Conv2d):
                target_layer = sublayer
                break
        if target_layer:
            break

    if target_layer is None:
        print("Geen geschikte Conv2d-layer gevonden.")
        return
    else:
        print(f"Geselecteerde target layer: {target_layer}")

    # 5. Grad-CAM uitvoeren
    cam = GradCAM(model=model.model, target_layers=[target_layer])
    grayscale_cam = cam(input_tensor=input_tensor, targets=[target])[0]

    print("CAM min:", grayscale_cam.min())
    print("CAM max:", grayscale_cam.max())

    # 6. Overlay visualiseren
    cam_image = show_cam_on_image(img_resized / 255.0, grayscale_cam, use_rgb=True)
    plt.imshow(cam_image)
    plt.axis("off")
    plt.title("Grad-CAM voor YOLOv12 (automatisch geselecteerde layer)")
    plt.show()

# ------------------------
# Startpunt
# ------------------------
run_grad_cam(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1337.jpg")


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torch.nn import Conv2d

def load_image(path, size=640):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (size, size))
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
    return img_rgb, img_resized, img_tensor.unsqueeze(0)

def run_feature_map_vis(img_path):
    model = YOLO("YOLOv12mv8.pt")
    model.eval()

    img_rgb, img_resized, input_tensor = load_image(img_path)
    input_tensor = input_tensor.to(model.device)

    # 1. Zoek een diepe conv-laag
    target_layer = None
    for layer in reversed(model.model.model[:-1]):
        for sublayer in layer.modules():
            if isinstance(sublayer, Conv2d):
                target_layer = sublayer
                break
        if target_layer:
            break

    if target_layer is None:
        print("Geen geschikte Conv-layer gevonden.")
        return

    # 2. Registreer hook
    activations = []

    def hook_fn(module, input, output):
        activations.append(output.detach().cpu())

    hook_handle = target_layer.register_forward_hook(hook_fn)

    # 3. Run forward pass
    with torch.no_grad():
        _ = model.model(input_tensor)

    hook_handle.remove()

    if not activations:
        print("Geen activaties vastgelegd.")
        return

    # 4. Visualiseer gemiddelde feature map
    fmap = activations[0].squeeze(0)  # shape: [C, H, W]
    fmap_mean = fmap.mean(dim=0).numpy()
    fmap_norm = (fmap_mean - fmap_mean.min()) / (fmap_mean.max() - fmap_mean.min() + 1e-8)
    fmap_resized = cv2.resize(fmap_norm, (img_resized.shape[1], img_resized.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(fmap_resized * 255), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img_resized, 0.6, heatmap, 0.4, 0)

    plt.imshow(overlay)
    plt.axis("off")
    plt.title("Feature map activatie overlay")
    plt.show()

# ▶Start
run_feature_map_vis(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1337.jpg")


In [ ]:
model = YOLO("YOLOv12mv8.pt")

In [ ]:
%matplotlib inline
#  Voorspel (lage confidence threshold)
results = model.predict(source=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1337.jpg", conf=0.6, save=False, show=False)

# Debug: zijn er boxes?
if results[0].boxes:
    print(f"Gevonden: {len(results[0].boxes)} object(en)")

    # Annotated afbeelding ophalen
    img = results[0].plot()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Laat het zien
    plt.imshow(img)
    plt.axis("off")
    plt.show()
else:
    print("Geen objecten gedetecteerd.")

In [ ]:
run_feature_map_vis(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1287.jpg")

In [ ]:
%matplotlib inline
#  Voorspel (lage confidence threshold)
results = model.predict(source=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1287.jpg", conf=0.6, save=False, show=False)

# Debug: zijn er boxes?
if results[0].boxes:
    print(f"Gevonden: {len(results[0].boxes)} object(en)")

    # Annotated afbeelding ophalen
    img = results[0].plot()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Laat het zien
    plt.imshow(img)
    plt.axis("off")
    plt.show()
else:
    print("Geen objecten gedetecteerd.")

In [ ]:
run_feature_map_vis(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1353.jpg")

In [ ]:
%matplotlib inline
#  Voorspel (lage confidence threshold)
results = model.predict(source=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Foto's\IMG_1353.jpg", conf=0.6, save=False, show=False)

# Debug: zijn er boxes?
if results[0].boxes:
    print(f"Gevonden: {len(results[0].boxes)} object(en)")

    # Annotated afbeelding ophalen
    img = results[0].plot()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Laat het zien
    plt.imshow(img)
    plt.axis("off")
    plt.show()
else:
    print("Geen objecten gedetecteerd.")